In [1]:
import pandas as pd

## 此处为断点重跑的2份数据

In [2]:
dt = pd.concat([pd.read_csv("../data/follows_deep3_self2.csv"),pd.read_excel("../data/follows_deep3_self.xlsx")])

In [3]:
dt.rename(columns={"mid":"up","from":"user"},inplace=True)

In [4]:
dt.up = dt.up.astype(int)

In [5]:
dt.columns

Index(['Unnamed: 0', 'up', 'attribute', 'mtime', 'tag', 'special',
       'contract_info', 'uname', 'face', 'sign', 'official_verify', 'vip',
       'user'],
      dtype='object')

In [6]:
len(dt["user"].unique()),len(dt["up"].unique())

(645, 28196)

In [7]:
dt_sim = dt[["user","up"]]

In [8]:
matrix_user2up = dt_sim.groupby("user").agg(list)

In [9]:
matrix_user2up

,up
user,
1,"[335115, 2, 12, 9099524, 867152]"
2,"[1868902080, 623512391, 43855, 520811440, 3941..."
4,"[9617619, 7584632, 2026561407, 22721144, 48163..."
9,"[2223856, 698750420, 645456648, 282994, 187100..."
12,"[7788379, 313530204, 14110780, 481774557, 2054..."
...,...
1202762767,"[208259, 487922923, 386043247, 208259, 4879229..."
1472906636,"[1825301994, 352111, 613345189, 1641117, 35336..."
1608941798,"[687084214, 384205461, 235555226, 473222648, 3..."


In [10]:
len(dt.up.unique())

28196

## 用户-UP主倒排表 

In [11]:
matrix_up_reverse = dt_sim.groupby("up").agg(list)

In [12]:
matrix_up_reverse

,user
up,
1,"[44974083, 307584, 800161, 2443068]"
2,"[44974083, 12, 1, 93855752, 18, 19, 13705279, ..."
4,"[9, 5483]"
6,"[362144307, 5483]"
9,"[12, 2, 4, 18, 20, 19, 75, 106, 109, 848008, 2..."
...,...
2128509951,"[7807024, 14110780, 109, 53456, 307584, 757403..."
2134572964,[2986947]
2138676495,"[55775966, 5563350, 21857, 1506, 54191665, 141..."


In [13]:
users = dt["user"].unique()

In [14]:
ups = dt["up"].unique()

## 用户-用户相交矩阵(重复关注UP主累计)

In [15]:
from itertools import combinations,chain

In [16]:
from tqdm import tqdm

In [17]:
matrix_user2user = pd.DataFrame(index=users,columns=users)
matrix_user2user.loc[:,:] = 0
for up,user in tqdm(zip(matrix_up_reverse.index,matrix_up_reverse.user)):
    for i,j in combinations(user,2):
        matrix_user2user.loc[i,j] += 1
        matrix_user2user.loc[j,i] += 1

28196it [04:23, 106.90it/s]


In [18]:
users[:20]

array([    81922,  44974083,         9,        12,  31113228,  77021198,
         6701071,        16,  37781521,   5242903,         1,         2,
       369750017,         4,  93855752, 382820360,        18,   2371603,
              20,    745493], dtype=int64)

In [19]:
matrix_user2up["up_count"] = matrix_user2up["up"].apply(len)

In [20]:
matrix_user2up

,up,up_count
user,,
1,"[335115, 2, 12, 9099524, 867152]",5
2,"[1868902080, 623512391, 43855, 520811440, 3941...",448
4,"[9617619, 7584632, 2026561407, 22721144, 48163...",66
9,"[2223856, 698750420, 645456648, 282994, 187100...",250
12,"[7788379, 313530204, 14110780, 481774557, 2054...",214
...,...,...
1202762767,"[208259, 487922923, 386043247, 208259, 4879229...",6
1472906636,"[1825301994, 352111, 613345189, 1641117, 35336...",91
1608941798,"[687084214, 384205461, 235555226, 473222648, 3...",5


In [21]:
users[:50]

array([    81922,  44974083,         9,        12,  31113228,  77021198,
         6701071,        16,  37781521,   5242903,         1,         2,
       369750017,         4,  93855752, 382820360,        18,   2371603,
              20,    745493,        19,   4034583, 495419424,    606243,
        12824612,  97177641,  30949419,   2666541,  50212909, 477655087,
         7807024,  40857648,  33353778,  28278842,    872507,  14110780,
        38965308,  49246269,  13705279,    274491, 346165316,        75,
        20951119, 144900177,   1318997,    610390, 485703766,  35225688,
          782425,    913497], dtype=int64)

## 用户间相似度(jaccard)

In [22]:
def jaccard(user2up,user2user,user_a,user_b):
    return user2user[user_a][user_b]/len(set(user2up["up"][user_a])|set(user2up["up"][user_b])) if user2user.loc[user_a,user_b] else 0

In [23]:
jaccard(matrix_user2up,matrix_user2user,44974083,13705279)

0.014285714285714285

## 用户-用户相交矩阵(相似度) 

In [24]:
sim_user2user = pd.DataFrame(index=users,columns=users)

In [25]:
sim_user2user.loc[:,:] = 0

In [26]:
for user_a,user_b in tqdm(combinations(users,2)):
    sim = jaccard(matrix_user2up,matrix_user2user,user_a,user_b)
    sim_user2user[user_a][user_b] = sim
    sim_user2user[user_b][user_a] = sim

207690it [00:51, 4029.71it/s]


In [27]:
sim_user2user

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,8372353,99573889,14672002,346079366,477782158,314466447,10010775,2678945,514687139,73842852
81922,0,0.022989,0.00678,0.015564,0.019231,0,0.029557,0,0.013652,0.00813,...,0,0.013746,0.024096,0,0.017544,0,0.009009,0.012195,0,0
44974083,0.022989,0,0.018717,0.026786,0.006803,0.005495,0.050179,0.007143,0.013298,0,...,0.006536,0.007979,0.024242,0.007246,0.007092,0,0.010309,0.018293,0,0.016484
9,0.00678,0.018717,0,0.165829,0.029777,0.003322,0.037783,0,0.014199,0,...,0.00738,0.006061,0.010526,0,0.003846,0.013158,0.00639,0.017794,0,0.0033
12,0.015564,0.026786,0.165829,0,0.049861,0.007576,0.053221,0,0.015317,0,...,0.017167,0.008734,0.008,0,0,0.018727,0.01087,0.033058,0,0.003745
31113228,0.019231,0.006803,0.029777,0.049861,0,0,0.021875,0,0.009732,0,...,0,0.002427,0.00495,0,0,0.004505,0.008772,0.020305,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314466447,0,0,0.013158,0.018727,0.004505,0.009174,0.009174,0,0.003257,0,...,0,0.003279,0,0,0,0,0,0.021739,0,0
10010775,0.009009,0.010309,0.00639,0.01087,0.008772,0,0.008889,0,0.00639,0,...,0.011494,0.003205,0.009804,0,0.013333,0,0,0.01,0.015385,0.008475
2678945,0.012195,0.018293,0.017794,0.033058,0.020305,0.023256,0.015385,0,0.014184,0,...,0.017241,0.003534,0.042254,0.023256,0.021739,0.021739,0.01,0,0,0.011236
514687139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.090909,0,0.015385,0,0,0.018519


## 用户-对UP主的兴趣度

In [28]:
ups

array([326499679, 316568752,   9617619, ..., 551888585, 519620645,
        32068534])

In [29]:
list(ups[789:800])

[7633151,
 2394319,
 266876176,
 33881729,
 1848018,
 382651856,
 346165316,
 488423046,
 813818,
 486969127,
 14269054]

In [30]:
sim_user2user[sim_user2user[50952087]>0][50952087]

81922        0.017857
44974083     0.014388
31113228     0.017442
77021198     0.016393
6701071      0.011765
               ...   
14672002     0.021277
346079366    0.058824
477782158    0.105263
2678945      0.045455
73842852     0.015873
Name: 50952087, Length: 430, dtype: object

In [31]:
cache = {}

In [32]:
class Interest(object):
    def __init__(self,up_reverse,sim_user2user):
        self.up_reverse = up_reverse
        self.sim_user2user = sim_user2user
        self.cache = {}
    def interest_user2up(self,user,up):
        cache_value = self.cache.get((user,up))
        if cache_value:
            return cache_value
        
        interest = 0
        for other_user in self.up_reverse.loc[up,"user"]:
            interest += self.sim_user2user.loc[user,other_user]
        self.cache[(user,up)] = interest
        return interest

In [33]:
ups[123:134]

array([413535812, 335505768, 135980509, 278093107, 477655087, 472547659,
         3589472,  28672699, 451066708,  62594335, 403928672])

In [34]:
interests = Interest(matrix_up_reverse,sim_user2user) 

In [35]:
interests.interest_user2up(50952087,318240083)

0.0392156862745098

## 用户-K个最相似用户-对UP主关注度-关注度排序-推荐topN个UP主

In [36]:
sim_user2user[351502018].sort_values(ascending=False)

6574487      0.121951
520155988    0.117647
28278842     0.059113
669249689    0.057971
13354765     0.053191
               ...   
6055289             0
210291              0
26108626            0
35857776            0
430518565           0
Name: 351502018, Length: 645, dtype: object

In [37]:
sim_user2user_rank = sim_user2user.rank(ascending=False,method="max")

In [38]:
sim_user2user_rank = sim_user2user_rank.astype(int)

In [39]:
sim_user2user_rank["top3_user"] = sim_user2user_rank.apply(lambda x: list(x[x<3].index),axis=1)

In [40]:
sim_user2user_rank["top3_user_len"] = sim_user2user_rank["top3_user"].apply(len)

In [41]:
sim_user2user_rank

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,14672002,346079366,477782158,314466447,10010775,2678945,514687139,73842852,top3_user,top3_user_len
81922,645,90,425,344,221,645,124,645,265,43,...,65,645,174,645,191,414,645,645,[31700507],1
44974083,113,645,256,235,372,353,26,46,269,645,...,63,207,318,645,152,332,645,158,"[93855752, 5899551, 841451]",3
9,411,139,645,5,132,417,61,645,258,645,...,280,645,381,87,267,348,645,422,[41683],1
12,223,64,1,645,48,301,16,645,231,645,...,332,645,645,41,142,148,645,389,"[9, 2, 109, 23085689, 480776951, 7656665, 2165...",9
31113228,158,423,161,116,645,645,214,645,342,645,...,396,645,645,221,201,303,645,645,[8655125],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314466447,645,645,330,311,424,271,401,645,496,645,...,645,645,645,645,645,275,645,645,"[10949920, 466272]",2
10010775,351,309,434,387,352,645,409,645,421,645,...,294,645,216,645,645,456,77,308,[215073099],1
2678945,283,146,269,197,203,72,299,645,259,645,...,13,96,139,31,157,645,645,245,"[1532165, 6627715, 244241492]",3
514687139,645,645,645,645,645,645,645,645,645,645,...,645,645,26,645,51,645,645,123,"[1318997, 345630501]",2


In [42]:
sim_user2user_rank["top3_user_all_ups"] = sim_user2user_rank["top3_user"].apply(lambda x: list(chain(*matrix_user2up.loc[x,"up"])))
sim_user2user_rank["top3_user_all_ups_len"] = sim_user2user_rank["top3_user_all_ups"].apply(len)

In [43]:
sim_user2user_rank

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,477782158,314466447,10010775,2678945,514687139,73842852,top3_user,top3_user_len,top3_user_all_ups,top3_user_all_ups_len
81922,645,90,425,344,221,645,124,645,265,43,...,174,645,191,414,645,645,[31700507],1,"[81976, 285318902, 16836724, 271442527, 388742...",21
44974083,113,645,256,235,372,353,26,46,269,645,...,318,645,152,332,645,158,"[93855752, 5899551, 841451]",3,"[693340454, 233114659, 22135876, 22723402, 646...",380
9,411,139,645,5,132,417,61,645,258,645,...,381,87,267,348,645,422,[41683],1,"[405285618, 55775966, 96241951, 346327646, 282...",13
12,223,64,1,645,48,301,16,645,231,645,...,645,41,142,148,645,389,"[9, 2, 109, 23085689, 480776951, 7656665, 2165...",9,"[2223856, 698750420, 645456648, 282994, 187100...",2226
31113228,158,423,161,116,645,645,214,645,342,645,...,645,221,201,303,645,645,[8655125],1,"[7936287, 13854759, 702509335, 12546526, 10276...",181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314466447,645,645,330,311,424,271,401,645,496,645,...,645,645,645,275,645,645,"[10949920, 466272]",2,"[356537312, 549406178, 285227589, 431615976, 1...",22
10010775,351,309,434,387,352,645,409,645,421,645,...,216,645,645,456,77,308,[215073099],1,"[414350632, 32780753, 895206, 17223352, 377268...",90
2678945,283,146,269,197,203,72,299,645,259,645,...,139,31,157,645,645,245,"[1532165, 6627715, 244241492]",3,"[431898258, 843122, 231048690, 1817554, 325987...",357
514687139,645,645,645,645,645,645,645,645,645,645,...,26,645,51,645,645,123,"[1318997, 345630501]",2,"[235555226, 6514909, 212535360, 242283, 808401...",23


In [44]:
sim_user2user_rank["interests"] = sim_user2user_rank.apply(lambda x:{up:interests.interest_user2up(x.name,up) for up in x.top3_user_all_ups},axis=1)

In [45]:
sim_user2user_rank["top10_interests"] = sim_user2user_rank.apply(lambda x: sorted(x.interests.items(),key=lambda item:item[1],reverse=True)[:10],axis=1)

In [46]:
sim_user2user_rank["top10_up"] = sim_user2user_rank["top10_interests"].apply(lambda x: [i[0] for i in x])

In [47]:
sim_user2user_rank.loc[2678945,"top10_up"]

[37090048,
 546195,
 326499679,
 168598,
 562197,
 174501086,
 295723,
 375375,
 15773384,
 52250]